<a href="https://colab.research.google.com/github/kevinhe107/CAPSTONE_Bert/blob/main/LSTM_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import keras
from keras.models import Model
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D

In [ ]:
#imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CAPSTONE/data/cleaned_headlines_1105.csv')

In [ ]:
data

,News_Title,Sentiment
0,"['applied', 'optoelectronics', 'aaoi', 'may', ...",bearish
1,"['applied', 'optoelectronics', 'announces', 'd...",neutral
2,"['insider', 'buying', 'applied', 'optoelectron...",bullish
3,"['applied', 'optoelectronics', 'stock', 'tanke...",bearish
4,"['applied', 'optoelectronics', 'inc', 'aaoi', ...",neutral
...,...,...
1099,"['aehr', 'test', 'system', 'participate', 'ann...",neutral
1100,"['aehr', 'secures', 'order', 'system', 'new', ...",bullish
1101,"['aehr', 'announces', 'shipment', 'new', 'syst...",neutral
1102,"['aehr', 'present', 'annual', 'ld', 'micro', '...",neutral


In [ ]:
data.isnull().sum()

News_Title    0
Sentiment     0
dtype: int64

In [ ]:
data['Sentiment'].value_counts()

bullish    528
neutral    389
bearish    187
Name: Sentiment, dtype: int64

* tokenization

In [ ]:
headlines = np.array(data['News_Title'])
y = np.array(list(map(lambda x: 1 if x =='bullish' else (-1 if x == 'bearish' else 0), data['Sentiment'])))

In [ ]:
(unique, counts) = np.unique(y, return_counts=True)
counts

array([187, 389, 528])

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(headlines, y, test_size=0.2, random_state = 45)

In [ ]:
tokenizer = Tokenizer(num_words=1104 )
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

* Glove embedding vectorization of words

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
      
  return word_to_vec_map

In [ ]:
glove_path = '/content/drive/MyDrive/CAPSTONE/glove.6B.50d.txt'

In [ ]:
word_to_vec_map = read_glove_vector('/content/drive/MyDrive/CAPSTONE/glove.6B.50d.txt')

maxLen = 150

In [ ]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)


In [ ]:
def lstm_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.5)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.5)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.5)(X)

  X = LSTM(256)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='relu')(embeddings)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='relu')(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
model = lstm_model((maxLen,))
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           85950     
_________________________________________________________________
lstm_6 (LSTM)                (None, 150, 128)          91648     
_________________________________________________________________
dropout_7 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 150, 128)          131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 256)               3942

In [ ]:
model_1d = conv1d_model((maxLen,))
model_1d.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           85950     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 148, 512)          77312     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 49, 512)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 47, 256)           393472    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 15, 256)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 13, 256)           1968

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
14/14 [==============================] - 4s 34ms/step - loss: 1.0338 - accuracy: 0.3364
Epoch 2/15
14/14 [==============================] - 0s 33ms/step - loss: 0.6298 - accuracy: 0.3610
Epoch 3/15
14/14 [==============================] - 0s 34ms/step - loss: 0.6462 - accuracy: 0.3324
Epoch 4/15
14/14 [==============================] - 0s 34ms/step - loss: 0.6279 - accuracy: 0.3616
Epoch 5/15
14/14 [==============================] - 0s 34ms/step - loss: 0.6483 - accuracy: 0.3498
Epoch 6/15
14/14 [==============================] - 0s 34ms/step - loss: 0.6413 - accuracy: 0.3747
Epoch 7/15
14/14 [==============================] - 0s 34ms/step - loss: 0.6357 - accuracy: 0.3811
Epoch 8/15
14/14 [==============================] - 0s 34ms/step - loss: 0.6117 - accuracy: 0.3805
Epoch 9/15
14/14 [==============================] - 0s 34ms/step - loss: 0.6288 - accuracy: 0.3626
Epoch 10/15
14/14 [==============================] - 0s 34ms/step - loss: 0.6205 - accuracy: 0.3785
Epoch 11/

In [ ]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
model.evaluate(X_test_indices, Y_test)

7/7 [==============================] - 1s 13ms/step - loss: 0.5947 - accuracy: 0.3575


[0.5947369337081909, 0.35746607184410095]

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
14/14 [==============================] - 1s 8ms/step - loss: 0.6931 - accuracy: 0.3403
Epoch 2/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6929 - accuracy: 0.3321
Epoch 3/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6926 - accuracy: 0.3712
Epoch 4/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6925 - accuracy: 0.3467
Epoch 5/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6921 - accuracy: 0.3400
Epoch 6/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6919 - accuracy: 0.3714
Epoch 7/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6919 - accuracy: 0.3166
Epoch 8/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6914 - accuracy: 0.3713
Epoch 9/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6913 - accuracy: 0.3345
Epoch 10/15
14/14 [==============================] - 0s 7ms/step - loss: 0.6912 - accuracy: 0.3387
Epoch 11/15
14/14 [

In [ ]:
model_1d.evaluate(X_test_indices, Y_test)

7/7 [==============================] - 0s 3ms/step - loss: 0.6890 - accuracy: 0.3575


[0.6890302896499634, 0.35746607184410095]

In [ ]:
preds_1d = model_1d.predict(X_test_indices)

In [ ]:
preds = model.predict(X_test_indices)

In [ ]:
X_test.shape

(221,)

In [ ]:
n = np.random.randint(0,221)

X_test[n]

"['zacks', 'analyst', 'blog', 'highlight', 'generac', 'applied', 'material', 'google', 'water', 'corp', 'adobe']"

In [ ]:
if preds[n] > 0.5:
  print('predicted sentiment : bullish')
elif preds[n]<0.5 and preds[n]>-0.5:
  print('predicted sentiment : neutral')
else: 
  print('precicted sentiment : bearish')

if (Y_test[n] == 1):
  print('correct sentiment : bullish')
elif (Y_test[n] == 0):
  print('correct sentiment : neutral')
else:
  print('correct sentiment : bearish')

predicted sentiment : neutral
correct sentiment : neutral


In [ ]:
preds[n]

array([0.30355722], dtype=float32)

In [ ]:
Y_test[n]

0

In [ ]:
headlines_idx = tokenizer.texts_to_sequences(headlines)

In [ ]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  headlines_idx_pad = pad_sequences(headlines_idx, maxlen=maxLen, padding='post')

  headlines_preds = model.predict(headlines_idx_pad)

  data['sentiment score'] = headlines_preds

  pred_sentiment = np.array(list(map(lambda x : 'bullish' if x > 0.5 else ('neutral' if x < 0.5 and x > -0.5  else 'bearish'), headlines_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data

In [ ]:
data

,News_Title,Sentiment,sentiment score,predicted sentiment
0,"['applied', 'optoelectronics', 'aaoi', 'may', ...",bearish,0.297649,neutral
1,"['applied', 'optoelectronics', 'announces', 'd...",neutral,0.297649,neutral
2,"['insider', 'buying', 'applied', 'optoelectron...",bullish,0.297649,neutral
3,"['applied', 'optoelectronics', 'stock', 'tanke...",bearish,0.297649,neutral
4,"['applied', 'optoelectronics', 'inc', 'aaoi', ...",neutral,0.297649,neutral
...,...,...,...,...
1099,"['aehr', 'test', 'system', 'participate', 'ann...",neutral,0.297649,neutral
1100,"['aehr', 'secures', 'order', 'system', 'new', ...",bullish,0.297649,neutral
1101,"['aehr', 'announces', 'shipment', 'new', 'syst...",neutral,0.297649,neutral
1102,"['aehr', 'present', 'annual', 'ld', 'micro', '...",neutral,0.297649,neutral


In [ ]:
data = add_score_predictions(data, headlines_idx)
data['predicted sentiment'].value_counts()

neutral    1104
Name: predicted sentiment, dtype: int64